In [47]:
import pandas as pd 

In [48]:
Anos_de_vida_saludable_a_los_65_anos = pd.read_excel('data\Anos_de_vida_saludable_a_los_65_anos.xls')

In [49]:
Anos_de_vida_saludable_a_los_65_anos.tail(3).isna().sum()/len(Anos_de_vida_saludable_a_los_65_anos) *100

Años de vida saludable a los 65 años     3.846154
Unnamed: 1                              11.538462
Unnamed: 2                              11.538462
Unnamed: 3                              11.538462
Unnamed: 4                              11.538462
Unnamed: 5                              11.538462
Unnamed: 6                              11.538462
Unnamed: 7                              11.538462
Unnamed: 8                              11.538462
Unnamed: 9                              11.538462
Unnamed: 10                             11.538462
Unnamed: 11                             11.538462
Unnamed: 12                             11.538462
Unnamed: 13                             11.538462
Unnamed: 14                             11.538462
Unnamed: 15                             11.538462
dtype: float64

In [50]:
# FUNCION DE LIMPIEZA GENERAL:

def limpieza_columnas(df):
    
    df_copy = df.copy()
    
    years = df_copy.iloc[1].values
    genders = df_copy.iloc[2].values
    
    # Crear los nuevos nombres de las columnas
    new_columns = []
    current_year = None
    for i in range(len(years)):
        if pd.isna(years[i]):
            new_columns.append(f'{current_year}_{genders[i]}')
        else:
            current_year = years[i]
            new_columns.append(f'{current_year}_{genders[i]}')
    
    df_copy.columns = new_columns
    
    df_copy = df_copy.iloc[3:]
    
    df_copy.set_index(df_copy.columns[0], inplace=True)
    
    # Eliminar las tres últimas filas si tienen más de un 90% de valores nulos, hay algunos excel que tienen nulos al final como observaciones
    rows_to_drop = df_copy.tail(3).index[df_copy.tail(3).isnull().mean(axis=1) > 0.1]
    df_copy = df_copy.drop(rows_to_drop)
    
    df_copy = df.reset_index()
    
    return df_copy

test = limpieza_columnas(Anos_de_vida_saludable_a_los_65_anos)

In [51]:
test

,index,Años de vida saludable a los 65 años,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,0,CC.AA.,Años,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,2017,NaN,NaN,2018,NaN,NaN,2019,NaN,NaN,2020,NaN,NaN,2021,NaN,NaN
2,2,NaN,Hombres,Mujeres,Total,Hombres,Mujeres,Total,Hombres,Mujeres,Total,Hombres,Mujeres,Total,Hombres,Mujeres,Total
3,3,España (ES),17.3,19.82,18.62,17.31,19.87,18.65,17.83,20.23,19.09,16.64,19.09,17.92,17.36,19.92,18.7
4,4,Andalucía (AN),15.67,16.66,16.16,15.55,16.64,16.09,17.42,18.81,18.17,16.84,18.24,17.59,16.83,18.24,17.58
5,5,Aragón (AR),18.66,21.65,20.18,18.73,21.69,20.24,18.08,20.31,19.44,16.79,18.56,17.91,17.56,19.79,18.91
6,6,"Asturias, Principado de (AS)",16.99,17.67,17.28,16.91,17.76,17.29,16.84,20.1,18.58,15.98,19.32,17.74,16.81,19.77,18.4
7,7,"Balears, Illes (IB)",18.38,21.74,19.97,18.55,22.05,20.2,19.47,19.58,19.42,18.99,19.38,19.09,18.96,19.37,19.07
8,8,Canarias (CN),16.68,19.93,18.53,16.44,19.63,18.24,18.68,19.24,18.86,18.61,19.12,18.76,18.63,19.17,18.79
9,9,Cantabria (CB),18.44,20.11,18.91,18.17,20.01,18.72,16.88,19.55,18.3,16.49,18.9,17.77,16.95,19.64,18.37


In [55]:
test.drop(test.index[:3],inplace=True)

In [56]:
test

,index,Años de vida saludable a los 65 años,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
3,3,España (ES),17.3,19.82,18.62,17.31,19.87,18.65,17.83,20.23,19.09,16.64,19.09,17.92,17.36,19.92,18.7
4,4,Andalucía (AN),15.67,16.66,16.16,15.55,16.64,16.09,17.42,18.81,18.17,16.84,18.24,17.59,16.83,18.24,17.58
5,5,Aragón (AR),18.66,21.65,20.18,18.73,21.69,20.24,18.08,20.31,19.44,16.79,18.56,17.91,17.56,19.79,18.91
6,6,"Asturias, Principado de (AS)",16.99,17.67,17.28,16.91,17.76,17.29,16.84,20.1,18.58,15.98,19.32,17.74,16.81,19.77,18.4
7,7,"Balears, Illes (IB)",18.38,21.74,19.97,18.55,22.05,20.2,19.47,19.58,19.42,18.99,19.38,19.09,18.96,19.37,19.07
8,8,Canarias (CN),16.68,19.93,18.53,16.44,19.63,18.24,18.68,19.24,18.86,18.61,19.12,18.76,18.63,19.17,18.79
9,9,Cantabria (CB),18.44,20.11,18.91,18.17,20.01,18.72,16.88,19.55,18.3,16.49,18.9,17.77,16.95,19.64,18.37
10,10,Castilla y León (CL),17.86,22.71,20.29,17.95,22.53,20.27,18.14,22.1,20.12,16.46,20.18,18.28,17.81,22.09,19.92
11,11,Castilla - La Mancha (CM),17.51,20.42,18.98,17.9,20.7,19.32,18.73,21.72,20.27,16.25,19.47,17.86,17.97,21.52,19.77
12,12,Cataluña (CT),16.94,19.49,18.27,16.99,19.52,18.3,18.32,21.97,20.27,16.66,20.2,18.53,17.75,21.62,19.8
